### Feature selection--wrapper model
* This notebook iteratively select 50 most significant features out of the 907 numerical features using silhouette score of the K-means algorithm as criteria

In [2]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, pdist
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.cm as cm
from sklearn.decomposition import PCA
import time

In [2]:
# Use a smaller data set to save time
df = pd.read_csv('PHBsample14_sss.csv', low_memory=False)
# drop the column resulted from sampling of the original data set
df.drop('Unnamed: 0', axis=1, inplace=True)
# In order to run K-means, drop all the categoricald data for now.
df = df.select_dtypes(include=['float64', 'int64'])
# Impute missing values with means
df = df.fillna(df.mean())

In [3]:
X = df

In [4]:
n, m = X.shape[0], X.shape[1]
print(n, m)

59159 907


In [5]:
model_test = KMeans(n_clusters=7)
model_test.fit(X) 
pred_y=model_test.labels_

In [9]:
print("Clustered class labels:", "\n", pd.value_counts(pd.Series(pred_y)))

Clustered class labels: 
 0    15491
5    14160
4     8910
6     5925
3     5687
1     5313
2     3673
dtype: int64


In [10]:
X.columns

Index(['ValDate', 'IssDate', 'IssAgeALB', 'Dur', 'AttAge', 'JointInd', 'AV',
       'CSV', 'SCPeriod', 'WDtoDate',
       ...
       'Match4', 'tie3', 'HealthScore_C5', 'Surr', 'EligibleInd', 'WDResponse',
       'FirstEligQInd', 'UtilizationInd', 'WDModelFilterIn', 'PolNum_UW'],
      dtype='object', length=907)

In [15]:
X.head()

,ValDate,IssDate,IssAgeALB,Dur,AttAge,JointInd,AV,CSV,SCPeriod,WDtoDate,...,Match4,tie3,HealthScore_C5,Surr,EligibleInd,WDResponse,FirstEligQInd,UtilizationInd,WDModelFilterIn,PolNum_UW
0,16343.0,16104.0,65.0,1.0,65.8,0.0,448559.96,421076.98,5,0.00,...,1.0,64.859049,0.500000,0.0,1.0,0.0,0.0,0.0,1.0,294692
1,15613.0,14397.0,69.0,4.0,72.4,0.0,67321.31,64451.77,7,0.00,...,1.0,53.000000,0.869053,0.0,1.0,0.0,0.0,0.0,1.0,281394
2,16070.0,13518.0,55.0,7.0,62.6,0.0,301121.04,295758.92,7,56438.97,...,1.0,57.000000,0.500000,0.0,1.0,0.0,0.0,1.0,0.0,475776
3,16343.0,14419.0,53.0,6.0,58.3,0.0,187344.04,180762.56,7,0.00,...,1.0,104.000000,0.500000,0.0,1.0,0.0,0.0,0.0,1.0,288738
4,15613.0,15044.0,63.0,2.0,65.2,0.0,183155.51,171789.66,6,1845.38,...,0.0,64.859049,0.869053,0.0,1.0,0.0,0.0,1.0,1.0,15320


### Use silhouette score instead of inertia

In [5]:
#reference: https://gist.github.com/AlexandreAbraham/5544803#comments
from itertools import combinations
from sklearn.utils import check_random_state
from sklearn.metrics.pairwise import distance_metrics
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.externals.joblib import Parallel, delayed

def silhouette_samples_block(X, labels, metric='euclidean', n_jobs=1, **kwds):
    """Compute the Silhouette Coefficient for each sample.
    The Silhoeutte Coefficient is a measure of how well samples are clustered
    with samples that are similar to themselves. Clustering models with a high
    Silhouette Coefficient are said to be dense, where samples in the same
    cluster are similar to each other, and well separated, where samples in
    different clusters are not very similar to each other.
    The Silhouette Coefficient is calculated using the mean intra-cluster
    distance (a) and the mean nearest-cluster distance (b) for each sample.
    The Silhouette Coefficient for a sample is ``(b - a) / max(a, b)``.
    This function returns the Silhoeutte Coefficient for each sample.
    The best value is 1 and the worst value is -1. Values near 0 indicate
    overlapping clusters.
    Parameters
    ----------
    X : array [n_samples_a, n_features]
        Feature array.
    labels : array, shape = [n_samples]
             label values for each sample
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string, it must be one of the options
        allowed by metrics.pairwise.pairwise_distances. If X is the distance
        array itself, use "precomputed" as the metric.
    `**kwds` : optional keyword parameters
        Any further parameters are passed directly to the distance function.
        If using a scipy.spatial.distance metric, the parameters are still
        metric dependent. See the scipy docs for usage examples.
    Returns
    -------
    silhouette : array, shape = [n_samples]
        Silhouette Coefficient for each samples.
    References
    ----------
    Peter J. Rousseeuw (1987). "Silhouettes: a Graphical Aid to the
        Interpretation and Validation of Cluster Analysis". Computational
        and Applied Mathematics 20: 53-65. doi:10.1016/0377-0427(87)90125-7.
    http://en.wikipedia.org/wiki/Silhouette_(clustering)
    """
    A = _intra_cluster_distances_block(X, labels, metric, n_jobs=n_jobs,
                                       **kwds)
    B = _nearest_cluster_distance_block(X, labels, metric, n_jobs=n_jobs,
                                        **kwds)
    sil_samples = (B - A) / np.maximum(A, B)
    # nan values are for clusters of size 1, and should be 0
    return np.nan_to_num(sil_samples)


def _intra_cluster_distances_block_(subX, metric, **kwds):
    distances = pairwise_distances(subX, metric=metric, **kwds)
    return distances.sum(axis=1) / (distances.shape[0] - 1)


def _intra_cluster_distances_block(X, labels, metric, n_jobs=1, **kwds):
    """Calculate the mean intra-cluster distance for sample i.
    Parameters
    ----------
    X : array [n_samples_a, n_features]
        Feature array.
    labels : array, shape = [n_samples]
        label values for each sample
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string, it must be one of the options
        allowed by metrics.pairwise.pairwise_distances. If X is the distance
        array itself, use "precomputed" as the metric.
    `**kwds` : optional keyword parameters
        Any further parameters are passed directly to the distance function.
        If using a scipy.spatial.distance metric, the parameters are still
        metric dependent. See the scipy docs for usage examples.
    Returns
    -------
    a : array [n_samples_a]
        Mean intra-cluster distance
    """
    intra_dist = np.zeros(labels.size, dtype=float)
    values = Parallel(n_jobs=n_jobs)(
            delayed(_intra_cluster_distances_block_)
                (X[np.where(labels == label)[0]], metric, **kwds)
                for label in np.unique(labels))
    for label, values_ in zip(np.unique(labels), values):
        intra_dist[np.where(labels == label)[0]] = values_
    return intra_dist


def _nearest_cluster_distance_block_(subX_a, subX_b, metric, **kwds):
    dist = pairwise_distances(subX_a, subX_b, metric=metric, **kwds)
    dist_a = dist.mean(axis=1)
    dist_b = dist.mean(axis=0)
    return dist_a, dist_b


def _nearest_cluster_distance_block(X, labels, metric, n_jobs=1, **kwds):
    """Calculate the mean nearest-cluster distance for sample i.
    Parameters
    ----------
    X : array [n_samples_a, n_features]
        Feature array.
    labels : array, shape = [n_samples]
        label values for each sample
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string, it must be one of the options
        allowed by metrics.pairwise.pairwise_distances. If X is the distance
        array itself, use "precomputed" as the metric.
    `**kwds` : optional keyword parameters
        Any further parameters are passed directly to the distance function.
        If using a scipy.spatial.distance metric, the parameters are still
        metric dependent. See the scipy docs for usage examples.
    X : array [n_samples_a, n_features]
        Feature array.
    Returns
    -------
    b : float
        Mean nearest-cluster distance for sample i
    """
    inter_dist = np.empty(labels.size, dtype=float)
    inter_dist.fill(np.inf)
    # Compute cluster distance between pairs of clusters
    unique_labels = np.unique(labels)

    values = Parallel(n_jobs=n_jobs)(
            delayed(_nearest_cluster_distance_block_)(
                X[np.where(labels == label_a)[0]],
                X[np.where(labels == label_b)[0]],
                metric, **kwds)
                for label_a, label_b in combinations(unique_labels, 2))

    for (label_a, label_b), (values_a, values_b) in \
            zip(combinations(unique_labels, 2), values):

            indices_a = np.where(labels == label_a)[0]
            inter_dist[indices_a] = np.minimum(values_a, inter_dist[indices_a])
            del indices_a
            indices_b = np.where(labels == label_b)[0]
            inter_dist[indices_b] = np.minimum(values_b, inter_dist[indices_b])
            del indices_b
    return inter_dist

### 25 features

In [ ]:
start = time.time()

# let's assume there are 7 clusters
num_of_cluster = 7
# Let's assume we're going to select 50 features out of 907 features, therefore we're going to iterate 50 times
num_of_iter = 50
model = KMeans(n_clusters=num_of_cluster)
score = np.zeros([num_of_iter, m]) # the sum of squared distances of samples to their closest cluster center
exclude_columns = [] # best performed models with selected features will be added to this list after every iteration
include_columns = [i for i in range(np.shape(score)[1]) if i not in exclude_columns] # rest of the features

for iteration in range(num_of_iter):
    # The first iteration, we're going to test clustering models on each individual variables
    if iteration == 0:
        print("Now processing iteration %d" %iteration, "\n")   
        for i in range(m):
            data = X.iloc[:, i][:, np.newaxis]
            model.fit(data)
#             pred_y = model.labels_
#             print("cluster labels based on variable %s:" %X.columns[i], "\n", pd.value_counts(pd.Series(pred_y)))
#             
            
            score[iteration][i] = model.inertia_            
            
        selected_feature_index = np.argmin(score[iteration], axis=0) 
        selected_feature_score = np.amin(score[iteration], axis=0) 
        selected_feature = X[X.columns[selected_feature_index]]
        exclude_columns.append(selected_feature_index)
        print("Conclusion: cluster based on variable %s" %X.columns[selected_feature_index], "gives the best performance", "\n") 
    #for following iteration, we're going to add the rest the feature to the selected feature and perform cluster model
    else:
        print("Now processing iteration %d" %iteration, "\n") 
        for i in range(m):
            if i not in exclude_columns:
                # Generate data with features selected from last iteration plus each individual rest of the features
                data = pd.concat([selected_feature, X[X.columns[i]]], axis=1)
                model.fit(data)
#                 pred_y = model.labels_
#                 print("cluster labels based on variables:", data.columns, "\n", pd.value_counts(pd.Series(pred_y)))
 
                score[iteration][i] = silhouette_score(data, model.labels_, sample_size=3000)
        include_columns = [i for i in range(np.shape(score)[1]) if i not in exclude_columns]
        selected_feature_score = np.amax(score[:,include_columns][iteration], axis=0) 
        selected_feature_index = np.argmax(score[:,include_columns][iteration], axis=0) 
        selected_feature = pd.concat([selected_feature, X[X.columns[selected_feature_index]]], axis=1)
        exclude_columns.append(selected_feature_index)
        print("Conclusion: cluster based on variable %s" %X.columns[exclude_columns], "gives the best performance", "\n") 
print("Selected features are %s" %X.columns[exclude_columns])

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print('The whole process took:')
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))


Now processing iteration 0 

Conclusion: cluster based on variable JointInd gives the best performance 

Now processing iteration 1 



/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1'], dtype='object') gives the best performance 

Now processing iteration 2 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'PartialBaths_C1'], dtype='object') gives the best performance 

Now processing iteration 3 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'PartialBaths_C1', 'IssAgeALB'], dtype='object') gives the best performance 

Now processing iteration 4 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'PartialBaths_C1', 'IssAgeALB', 'Dur'], dtype='object') gives the best performance 

Now processing iteration 5 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'PartialBaths_C1', 'IssAgeALB', 'Dur',
       'IssAgeALB'],
      dtype='object') gives the best performance 

Now processing iteration 6 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'PartialBaths_C1', 'IssAgeALB', 'Dur',

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'PartialBaths_C1', 'IssAgeALB', 'Dur',
       'IssAgeALB', 'NoOfUnits_C1', 'BuildingClass_C1', 'NoOfCars_C1', 'CSV',
       'EstOutstandingLoanAmt_C1',
       'Population.16.Employed.Civilian.Occupation.Percent.ComputerMathematical_C3',
       'EstDwellValue_C1', 'imt42_C4', 'ihi01_C4', 'is004xam_C4', 'ifn43y5_C4',
       'iat89_C4', 'iat01_C4', 'iat112_C4', 'iin01_C4',
       'CEN_tr_pctLT10KAge65plus', 'CEN_tr_pctFinanceCon', 'icc01y5_C4',
       'ifn42_C4'],
      dtype='object') gives the best performance 

Now processing iteration 25 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'PartialBaths_C1', 'IssAgeALB', 'Dur',
       'IssAgeALB', 'NoOfUnits_C1', 'BuildingClass_C1', 'NoOfCars_C1', 'CSV',
       'EstOutstandingLoanAmt_C1',
       'Population.16.Employed.Civilian.Occupation.Percent.ComputerMathematical_C3',
       'EstDwellValue_C1', 'imt42_C4', 'ihi01_C4', 'is004xam_C4', 'ifn43

### 50 features

In [6]:
start = time.time()

# let's assume there are 7 clusters
num_of_cluster = 7
# Let's assume we're going to select 50 features out of 907 features, therefore we're going to iterate 50 times
num_of_iter = 50
model = KMeans(n_clusters=num_of_cluster)
score = np.zeros([num_of_iter, m]) # the sum of squared distances of samples to their closest cluster center
exclude_columns = [] # best performed models with selected features will be added to this list after every iteration
include_columns = [i for i in range(np.shape(score)[1]) if i not in exclude_columns] # rest of the features

for iteration in range(num_of_iter):
    # The first iteration, we're going to test clustering models on each individual variables
    if iteration == 0:
        print("Now processing iteration %d" %iteration, "\n")   
        for i in range(m):
            data = X.iloc[:, i][:, np.newaxis]
            model.fit(data)
#             pred_y = model.labels_
#             print("cluster labels based on variable %s:" %X.columns[i], "\n", pd.value_counts(pd.Series(pred_y)))
#             
            
            score[iteration][i] = model.inertia_            
            
        selected_feature_index = np.argmin(score[iteration], axis=0) 
        selected_feature_score = np.amin(score[iteration], axis=0) 
        selected_feature = X[X.columns[selected_feature_index]]
        exclude_columns.append(selected_feature_index)
        print("Conclusion: cluster based on variable %s" %X.columns[selected_feature_index], "gives the best performance", "\n") 
    #for following iteration, we're going to add the rest the feature to the selected feature and perform cluster model
    else:
        print("Now processing iteration %d" %iteration, "\n") 
        for i in range(m):
            if i not in exclude_columns:
                # Generate data with features selected from last iteration plus each individual rest of the features
                data = pd.concat([selected_feature, X[X.columns[i]]], axis=1)
                model.fit(data)
#                 pred_y = model.labels_
#                 print("cluster labels based on variables:", data.columns, "\n", pd.value_counts(pd.Series(pred_y)))
 
                score[iteration][i] = silhouette_score(data, model.labels_, sample_size=3000)
        include_columns = [i for i in range(np.shape(score)[1]) if i not in exclude_columns]
        selected_feature_score = np.amax(score[:,include_columns][iteration], axis=0) 
        selected_feature_index = np.argmax(score[:,include_columns][iteration], axis=0) 
        selected_feature = pd.concat([selected_feature, X[X.columns[selected_feature_index]]], axis=1)
        exclude_columns.append(selected_feature_index)
        print("Conclusion: cluster based on variable %s" %X.columns[exclude_columns], "gives the best performance", "\n") 
print("Selected features are %s" %X.columns[exclude_columns])

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print('The whole process took:')
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))


Now processing iteration 0 

Conclusion: cluster based on variable JointInd gives the best performance 

Now processing iteration 1 



/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1'], dtype='object') gives the best performance 

Now processing iteration 2 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'NoOfBuildings_C1'], dtype='object') gives the best performance 

Now processing iteration 3 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'NoOfBuildings_C1', 'WDtoDate'], dtype='object') gives the best performance 

Now processing iteration 4 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'NoOfBuildings_C1', 'WDtoDate',
       'MaxPayment_C1'],
      dtype='object') gives the best performance 

Now processing iteration 5 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'NoOfBuildings_C1', 'WDtoDate',
       'MaxPayment_C1', 'TotalPayments_C1'],
      dtype='object') gives the best performance 

Now processing iteration 6 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'NoOfBuildings_C1', 'WDtoDate',
       'MaxPayment_C1', 'TotalPayments_C1', 'EstOutstandingLoanAmt_C1',
       'Home.Equity.Loan.Date.YYYYMM_C3', 'TotalOrigAmtBorrowed_C1',
       'Living.Area.Square.Feet_num', 'CEN_tr_pctFarmingOcc', 'AVMChg1Yr_C1',
       'NoOfRooms_C1', 'OutstandingLoanToMarket_C1', 'i03ccac1_C4',
       'i03cctc1_C4', 'CEN_tr_pctCleaningOcc', 'imt07_C4',
       'CEN_bg_pctAgricultureIndustry', 'ibr02y5_C4', 'i12ccnl1_C4',
       'CEN_tr_age80plus', 'icc96m01_C4'],
      dtype='object') gives the best performance 

Now processing iteration 23 

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'NoOfBuildings_C1', 'WDtoDate',
       'MaxPayment_C1', 'TotalPayments_C1', 'EstOutstandingLoanAmt_C1',
       'Home.Equity.Loan.Date.YYYYMM_C3', 'TotalOrigAmtBorrowed_C1',
       'Living.Area.Square.Feet_num', 'CEN_tr_pctFarmingOcc', 'AVMChg1Yr_C1',
       'NoOfRooms_C1', 'Outstandin

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'NoOfBuildings_C1', 'WDtoDate',
       'MaxPayment_C1', 'TotalPayments_C1', 'EstOutstandingLoanAmt_C1',
       'Home.Equity.Loan.Date.YYYYMM_C3', 'TotalOrigAmtBorrowed_C1',
       'Living.Area.Square.Feet_num', 'CEN_tr_pctFarmingOcc', 'AVMChg1Yr_C1',
       'NoOfRooms_C1', 'OutstandingLoanToMarket_C1', 'i03ccac1_C4',
       'i03cctc1_C4', 'CEN_tr_pctCleaningOcc', 'imt07_C4',
       'CEN_bg_pctAgricultureIndustry', 'ibr02y5_C4', 'i12ccnl1_C4',
       'CEN_tr_age80plus', 'icc96m01_C4', 'i12ccsv1_C4', 'EstDwellValue_C1',
       'CEN_bg_pctGE200KAge65plus', 'CEN_tr_pctGE150KAge45plus', 'imt102_C4',
       'i12ccsu1_C4', 'ifn20_C4', 'CEN_tr_pctAssociateDegree', 'iat42_C4',
       'CEN_bg_pctHHincomeLT30K',
       'Population.16.Employed.Civilian.Occupation.Percent.ArchitectureEngineering_C3'],
      dtype='object') gives the best performance 

Now processing iteration 34 

Conclusion: cluster based on variable Index(['J

Conclusion: cluster based on variable Index(['JointInd', 'NoOfCars_C1', 'NoOfBuildings_C1', 'WDtoDate',
       'MaxPayment_C1', 'TotalPayments_C1', 'EstOutstandingLoanAmt_C1',
       'Home.Equity.Loan.Date.YYYYMM_C3', 'TotalOrigAmtBorrowed_C1',
       'Living.Area.Square.Feet_num', 'CEN_tr_pctFarmingOcc', 'AVMChg1Yr_C1',
       'NoOfRooms_C1', 'OutstandingLoanToMarket_C1', 'i03ccac1_C4',
       'i03cctc1_C4', 'CEN_tr_pctCleaningOcc', 'imt07_C4',
       'CEN_bg_pctAgricultureIndustry', 'ibr02y5_C4', 'i12ccnl1_C4',
       'CEN_tr_age80plus', 'icc96m01_C4', 'i12ccsv1_C4', 'EstDwellValue_C1',
       'CEN_bg_pctGE200KAge65plus', 'CEN_tr_pctGE150KAge45plus', 'imt102_C4',
       'i12ccsu1_C4', 'ifn20_C4', 'CEN_tr_pctAssociateDegree', 'iat42_C4',
       'CEN_bg_pctHHincomeLT30K',
       'Population.16.Employed.Civilian.Occupation.Percent.ArchitectureEngineering_C3',
       'PurchaseDt_C1', 'Death', 'Pol_TermDt', 'CEN_tr_pctHHInvestIncome',
       'AVPctEq', 'GMDBInd', 'DeptoDate', 'WDCount',
 

### Select features from the pre-selected 544 variables

In [3]:
# Use a smaller data set to save time
df = pd.read_csv('PHBsample14_sss.csv', low_memory=False)
# drop the column resulted from sampling of the original data set
df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
selected_variable = pd.read_csv('selectedVariables.csv')
selected_variable.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df1 = df[df.columns.intersection(selected_variable.columns)]

In [7]:
# Separate numerical and categorical data
df_num = df1.select_dtypes(include=['float64', 'int64'])
df_cat = df1.select_dtypes(include=['object'])
# Impute missing values with means
df_num = df_num.fillna(df_num.mean())
df_cat = pd.get_dummies(df_cat)

In [21]:
X = pd.concat([df_num, df_cat], axis=1)

In [9]:
n, m = X.shape[0], X.shape[1]
print(n, m)

59159 576


In [ ]:
start = time.time()

# let's assume there are 7 clusters
num_of_cluster = 7
# Let's assume we're going to select 50 features out of 544 features, therefore we're going to iterate 50 times
num_of_iter = 50
model = KMeans(n_clusters=num_of_cluster)
score = np.zeros([num_of_iter, m]) # the sum of squared distances of samples to their closest cluster center
exclude_columns = [] # best performed models with selected features will be added to this list after every iteration
include_columns = [i for i in range(np.shape(score)[1]) if i not in exclude_columns] # rest of the features

for iteration in range(num_of_iter):
    # The first iteration, we're going to test clustering models on each individual variables
    if iteration == 0:
        print("Now processing iteration %d" %iteration, "\n")   
        for i in range(m):
            data = X.iloc[:, i][:, np.newaxis]
            model.fit(data)
#             pred_y = model.labels_
#             print("cluster labels based on variable %s:" %X.columns[i], "\n", pd.value_counts(pd.Series(pred_y)))
#             
            
            score[iteration][i] = model.inertia_            
            
        selected_feature_index = np.argmin(score[iteration], axis=0) 
        selected_feature_score = np.amin(score[iteration], axis=0) 
        selected_feature = X[X.columns[selected_feature_index]]
        exclude_columns.append(selected_feature_index)
        print("Conclusion: cluster based on variable %s" %X.columns[selected_feature_index], "gives the best performance", "\n") 
    #for following iteration, we're going to add the rest the feature to the selected feature and perform cluster model
    else:
        print("Now processing iteration %d" %iteration, "\n") 
        for i in range(m):
            if i not in exclude_columns:
                # Generate data with features selected from last iteration plus each individual rest of the features
                data = pd.concat([selected_feature, X[X.columns[i]]], axis=1)
                model.fit(data)
#                 pred_y = model.labels_
#                 print("cluster labels based on variables:", data.columns, "\n", pd.value_counts(pd.Series(pred_y)))
 
                score[iteration][i] = silhouette_score(data, model.labels_, sample_size=3000)
        include_columns = [i for i in range(np.shape(score)[1]) if i not in exclude_columns]
        selected_feature_score = np.amax(score[:,include_columns][iteration], axis=0) 
        selected_feature_index = np.argmax(score[:,include_columns][iteration], axis=0) 
        selected_feature = pd.concat([selected_feature, X[X.columns[selected_feature_index]]], axis=1)
        exclude_columns.append(selected_feature_index)
        print("Conclusion: cluster based on variable %s" %X.columns[exclude_columns], "gives the best performance", "\n") 
print("Selected features are %s" %X.columns[exclude_columns])

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print('The whole process took:')
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))


Now processing iteration 0 

Conclusion: cluster based on variable JointInd gives the best performance 

Now processing iteration 1 



/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


Conclusion: cluster based on variable Index(['JointInd', 'Number.of.Active.Sources_C3'], dtype='object') gives the best performance 

Now processing iteration 2 

Conclusion: cluster based on variable Index(['JointInd', 'Number.of.Active.Sources_C3', 'CEN_tr_pctGE150KAge65plus'], dtype='object') gives the best performance 

Now processing iteration 3 

Conclusion: cluster based on variable Index(['JointInd', 'Number.of.Active.Sources_C3', 'CEN_tr_pctGE150KAge65plus',
       'CEN_tr_pctGE200KAge65plus'],
      dtype='object') gives the best performance 

Now processing iteration 4 

Conclusion: cluster based on variable Index(['JointInd', 'Number.of.Active.Sources_C3', 'CEN_tr_pctGE150KAge65plus',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus'],
      dtype='object') gives the best performance 

Now processing iteration 5 

Conclusion: cluster based on variable Index(['JointInd', 'Number.of.Active.Sources_C3', 'CEN_tr_pctGE150KAge65plus',
       'CEN_tr_pctGE200KAge65plu

Conclusion: cluster based on variable Index(['JointInd', 'Number.of.Active.Sources_C3', 'CEN_tr_pctGE150KAge65plus',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctProductionFamily', 'CEN_tr_pctConstructionFamily',
       'CEN_tr_pctSalesFamily', 'CEN_tr_pctServiceFamily',
       'CEN_tr_pctManagementFamily', 'CEN_tr_pctProductionGovt',
       'CEN_tr_pctConstructionGovt', 'WDtoDate', 'EstMarketValue_C1',
       'PurchasePrice_C1', 'Number.of.Adults.Indicator_C3_N', 'ihi21_C4',
       'CEN_tr_pctHealthPractitionersOcc', 'CEN_bg_AvgHHSizeRenterOccup'],
      dtype='object') gives the best performance 

Now processing iteration 21 

Conclusion: cluster based on variable Index(['JointInd', 'Number.of.Active.Sources_C3', 'CEN_tr_pctGE150KAge65plus',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctP

Conclusion: cluster based on variable Index(['JointInd', 'Number.of.Active.Sources_C3', 'CEN_tr_pctGE150KAge65plus',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctProductionFamily', 'CEN_tr_pctConstructionFamily',
       'CEN_tr_pctSalesFamily', 'CEN_tr_pctServiceFamily',
       'CEN_tr_pctManagementFamily', 'CEN_tr_pctProductionGovt',
       'CEN_tr_pctConstructionGovt', 'WDtoDate', 'EstMarketValue_C1',
       'PurchasePrice_C1', 'Number.of.Adults.Indicator_C3_N', 'ihi21_C4',
       'CEN_tr_pctHealthPractitionersOcc', 'CEN_bg_AvgHHSizeRenterOccup',
       'CEN_tr_pctMalenoWifeHH', 'ibr01_C4', 'iat112_C4',
       'Total.Mortgage.Amounts.in.1000s.of.Dollars_C3', 'SCPeriod',
       'CEN_tr_pctWorkforceForProfit', 'iin20xam_C4', 'irt40_C4',
       'OriginalOwner_C1_Y', 'CEN_bg_pctManufacturingIndustry'],
      dtype='object') gives the best performance 

Now processing iteration 31 

